In [ ]:
from top_down_sorting import *
from vis import *
import numpy as np
import pickle

In [ ]:
# Load raw data csr matrix
datafile = 'paper_example.pkl'
with open(datafile, "rb") as f:
    raw_csr : csr_matrix = pickle.load(f).tocsr()
print(type(raw_csr))
matrix_heatmap(raw_csr.toarray(), title='Raw data')

In [ ]:
# Build heirarchical cache
m, n = raw_csr.shape
print(f'Starting m, n = {m, n}')
min_n, max_n = 2, n

raw_weights = np.ones(shape=(m,))
sorter = TopDownSorting(raw_csr, raw_weights)

for H, W, _ in sorter.build_cache((min_n, max_n)):
    current_m, current_n = H.shape
    matrix_heatmap(H.toarray(), title=f'Cache at #bins={current_n}, #edges={current_m}')
matrix_heatmap(raw_csr.toarray(), title='Raw data')

In [ ]:
# Sanity check - highest resolution should lok like raw data

H_final = H.tocsr()
raw = raw_csr.tocsr()

#assert H_final.shape == raw.shape

H_rows = sorted(map(tuple, H_final.toarray()))
raw_rows = sorted(map(tuple, raw.toarray()))

H_sorted = np.array(H_rows)
raw_sorted = np.array(raw_rows)

matrix_heatmap(H_sorted, title="Sorted H")
matrix_heatmap(raw_sorted, title="Sorted Raw")
